In [62]:
from urllib.request import urlopen, Request
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas_datareader as pdr
import datetime as dt
from datetime import datetime

In [63]:
df=pd.read_csv('final_train_dataset.csv')
df.dropna()

,Unnamed: 0.1,Unnamed: 0,sno,DATE,HEADLINE,TICKER,COMPANY NAME,SECTOR,SUBSECTOR,SERIES,...,close,volume,adj close,avg High,avg low,avg open,avg close,avg volume,avg adj close,trend
0,0,0,3,15-02-2022,Religare Ent Says\n▶️Religare Finvest victim o...,RELIGARE,Religare Enterprises Limited,Finance - Investments,Finance - Investments,EQ,...,122.050003,122.050003,910705,159.109998,159.779999,152.860001,156.130000,156.130000,686379.6,bearish
1,1,1,4,15-02-2022,Ashok Leyland targets 40% ICV sales from CNG v...,ASHOKLEY,Ashok Leyland Limited,Auto - LCVs & HCVs,Auto - LCVs & HCVs,EQ,...,128.500000,127.602028,15896279,159.329999,161.720001,156.199997,158.689999,157.581058,16145476.0,bearish
2,2,2,6,15-02-2022,Cipla promoters plan to sell 2.5% of stock,CIPLA,Cipla Limited,Pharmaceuticals,Pharmaceuticals,EQ,...,921.849976,917.393127,8304774,1135.429993,1155.719995,1129.699988,1145.319995,1139.782751,3662772.4,bullish
3,3,3,7,15-02-2022,"2.05 cr shares of Cipla worth ₹1,880 cr trade ...",CIPLA,Cipla Limited,Pharmaceuticals,Pharmaceuticals,EQ,...,921.849976,917.393127,8304774,1135.429993,1155.719995,1129.699988,1145.319995,1139.782751,3662772.4,bullish
4,4,4,8,15-02-2022,NMDC in focus\n\nIron Ore falls 10% on SGX as ...,NMDC,NMDC Limited,Mining & Minerals,Mining & Minerals,EQ,...,149.300003,143.697586,17659797,186.379999,189.089999,182.490005,185.719998,178.750937,18851958.2,bullish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5764,5764,5764,9043,14-02-2023,I am proud to announce today the purchase of o...,IMPAL,India Motor Parts and Accessories Limited,Auto - 2 & 3 Wheelers,Auto - 2 & 3 Wheelers,EQ,...,713.250000,713.250000,317,858.229993,862.419995,849.059998,855.849988,852.264673,901.0,bearish
5765,5765,5765,9044,14-02-2023,Mcleod Russel India reports Q3 earnings.\n\n◼️...,IMPAL,India Motor Parts and Accessories Limited,Auto - 2 & 3 Wheelers,Auto - 2 & 3 Wheelers,EQ,...,713.250000,713.250000,317,858.229993,862.419995,849.059998,855.849988,852.264673,901.0,bearish
5766,5766,5766,9045,14-02-2023,Genesys International reports Q3 earnings.\n\n...,GENESYS,Genesys International Corporation Limited,Computers - Software Medium & Small,Computers - Software Medium & Small,EQ,...,463.500000,463.500000,39934,507.209998,528.680005,492.419995,516.640002,516.640002,29212.0,bearish
5768,5768,5768,9048,15-02-2023,*WHAT HAPPENED OVERNIGHT*\n\n- Stocks finish m...,OIL,Oil India Limited,Oil Drilling,Oil Drilling,EQ,...,257.799988,257.799988,12116861,276.589996,285.639999,274.030002,281.639999,281.639999,5440190.8,bearish


In [64]:
from sklearn.model_selection import train_test_split

In [65]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'sno', 'DATE', 'HEADLINE', 'TICKER',
       'COMPANY NAME', 'SECTOR', 'SUBSECTOR', 'SERIES', 'DATE OF LISTING',
       'PAID UP VALUE', 'MARKET LOT', 'ISIN NUMBER', 'FACE VALUE',
       'negative_score', 'positive_score', 'neutral_score', 'compound_score',
       'High', 'Low', 'Open', 'close', 'volume', 'adj close', 'avg High',
       'avg low', 'avg open', 'avg close', 'avg volume', 'avg adj close',
       'trend'],
      dtype='object')

In [66]:
X=df.drop('trend',axis=1)
y=df['trend']

In [67]:
X.head()

,Unnamed: 0.1,Unnamed: 0,sno,DATE,HEADLINE,TICKER,COMPANY NAME,SECTOR,SUBSECTOR,SERIES,...,Open,close,volume,adj close,avg High,avg low,avg open,avg close,avg volume,avg adj close
0,0,0,3,15-02-2022,Religare Ent Says\n▶️Religare Finvest victim o...,RELIGARE,Religare Enterprises Limited,Finance - Investments,Finance - Investments,EQ,...,120.750000,122.050003,122.050003,910705,159.109998,159.779999,152.860001,156.130000,156.130000,686379.6
1,1,1,4,15-02-2022,Ashok Leyland targets 40% ICV sales from CNG v...,ASHOKLEY,Ashok Leyland Limited,Auto - LCVs & HCVs,Auto - LCVs & HCVs,EQ,...,127.199997,128.500000,127.602028,15896279,159.329999,161.720001,156.199997,158.689999,157.581058,16145476.0
2,2,2,6,15-02-2022,Cipla promoters plan to sell 2.5% of stock,CIPLA,Cipla Limited,Pharmaceuticals,Pharmaceuticals,EQ,...,920.099976,921.849976,917.393127,8304774,1135.429993,1155.719995,1129.699988,1145.319995,1139.782751,3662772.4
3,3,3,7,15-02-2022,"2.05 cr shares of Cipla worth ₹1,880 cr trade ...",CIPLA,Cipla Limited,Pharmaceuticals,Pharmaceuticals,EQ,...,920.099976,921.849976,917.393127,8304774,1135.429993,1155.719995,1129.699988,1145.319995,1139.782751,3662772.4
4,4,4,8,15-02-2022,NMDC in focus\n\nIron Ore falls 10% on SGX as ...,NMDC,NMDC Limited,Mining & Minerals,Mining & Minerals,EQ,...,149.050003,149.300003,143.697586,17659797,186.379999,189.089999,182.490005,185.719998,178.750937,18851958.2


In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [69]:
X_train['trend']=y_train

In [70]:
X_train.head()

,Unnamed: 0.1,Unnamed: 0,sno,DATE,HEADLINE,TICKER,COMPANY NAME,SECTOR,SUBSECTOR,SERIES,...,close,volume,adj close,avg High,avg low,avg open,avg close,avg volume,avg adj close,trend
1902,1902,1902,3309,23-06-2022,Thermax subsidiary First Energy will acquire t...,ENERGYDEV,Energy Development Company Limited,Power,Power,EQ,...,15.100000,15.100000,55670,19.270000,19.510000,18.170000,18.570000,18.570000,112531.8,bearish
443,443,443,874,17-03-2022,Torrent Pharma enters into licensing agreement...,TORNTPHARM,Torrent Pharmaceuticals Limited,Pharmaceuticals,Pharmaceuticals,EQ,...,1437.300049,1416.654175,865328,1698.300000,1730.430005,1680.655005,1701.665015,1677.221729,391710.8,bearish
1084,1084,1084,1897,28-04-2022,Fineotex Chemical reports Q4 earnings.\n\n▶️Ne...,FCL,Fineotex Chemical Limited,NaN,NaN,EQ,...,206.899994,206.520279,524816,256.939996,264.990002,246.729999,252.820001,252.356003,1271654.8,bearish
37,37,37,82,18-02-2022,. @IndiGo6E takes delivery of its first aircra...,IMFA,Indian Metals & Ferro Alloys Limited,Steel - Pig Iron,Steel - Pig Iron,EQ,...,348.500000,325.614288,42703,424.500000,430.760010,413.900000,421.410004,393.736359,59315.2,bearish
3563,3563,3563,5952,28-09-2022,Smallcap World Fund acquires additional 2.63% ...,MASTEK,Mastek Limited,Computers - Software,Computers - Software,EQ,...,1711.900024,1697.216064,62260,2116.139990,2139.189990,2085.619995,2097.269995,2079.280469,114248.4,bullish


In [71]:
X_train.to_csv('training.csv')

In [72]:
X_test.to_csv('test.csv')
y_test.to_csv('test_result.csv')